# 1. Importamos librerias

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import nltk
from nltk import tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import pandas as pd
import numpy as np

import sklearn as sk

import seaborn as sns
import matplotlib.pyplot as plt
from six import StringIO
import pydotplus
from IPython.display import Image

from sklearn.tree import export_graphviz

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Importamos dataSet

In [ ]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_trabajo_train = df.copy()
df_trabajo_test = df_test.copy()

df_trabajo_train

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo
...,...,...,...
49995,49995,Pensé que esta película hizo un buen trabajo a...,positivo
49996,49996,"Mala parcela, mal diálogo, mala actuación, dir...",negativo
49997,49997,Soy católica enseñada en escuelas primarias pa...,negativo
49998,49998,Voy a tener que estar en desacuerdo con el com...,negativo


# Preprocesamiento

## Definimos una funcion para quitar letras con tildes

In [ ]:
def remove_tildes(text):
    remove = (("á", "a"),("é", "e"),("í", "i"),("ó", "o"),("ú", "u"))

    for a, b in remove:
        text = text.replace(a, b).replace(a.upper(), b.upper())
    return text

## Stop words
- Usaremos stopWords tanto en español como en ingles y les haremos un procesamiento para quitar las tildes.

In [ ]:
# Por recomendacion generales de la comunidad, quitaremos las tildes de las stop words.
stop_words = stopwords.words("english") + stopwords.words("spanish")
stop_words = [remove_tildes(word) for word in stop_words]

## Vectorizacion
Para vectorizar utilizaremos TfidfVectorizer ya que por indicaciones de la catedra y pruebas del equipo suele ofrecer una mejora en el rendimiento.

### Parametros de TfidfVectorizer:
1.   **max_features:** Este parámetro especifica el número máximo de características (términos o palabras) que se deben extraer y tener en cuenta. Por ejemplo, en 2000, significa que solo se considerarán las 2000 palabras más frecuentes en el conjunto de documentos. Probamos con utilizar diversos valores en este campo, (1000, 2500, 5000) y el mejor resultado lo conseguimos dejandolo con valores default. Sin embargo, despues de ciertas pruebas y mejoras, descubrimos que dejando sin limite esta caracteristica consumiamos toda la ram, por lo que se tuvo que limitar.

2.   **stop_words:** Este parámetro se utiliza para especificar una lista de palabras comunes que se deben omitir durante el proceso de vectorización. Las palabras "stop words" suelen ser palabras muy comunes en un idioma (como "the", "and", "is", etc.) que no aportan mucha información sobre el contenido del texto y pueden ser eliminadas para mejorar la eficiencia y la relevancia del análisis.

3. **strip_accents:** Este parámetro indica si se deben eliminar o reemplazar acentos y diacríticos durante el preprocesamiento del texto.

In [ ]:
vectorizer = TfidfVectorizer(max_features = 1000, stop_words = stop_words, strip_accents = "unicode")

## Definimos una funcion para quitar letras con tildes

In [ ]:
def remove_tildes(text):
    remove = (("á", "a"),("é", "e"),("í", "i"),("ó", "o"),("ú", "u"))

    for a, b in remove:
        text = text.replace(a, b).replace(a.upper(), b.upper())
    return text

# 3. Dividir conjunto train y test

In [ ]:
# Mapear los valores "positivo" a 1 y "negativo" a 0 en la columna 'Sentimiento'
df_trabajo_train['sentimiento'] = df_trabajo_train['sentimiento'].map({'positivo': 1, 'negativo': 0})
data = vectorizer.fit_transform(df_trabajo_train.review_es)
data = data.toarray()

x_train, x_test, y_train, y_test = train_test_split(data, df_trabajo_train.sentimiento, test_size = 0.30, random_state = 25, shuffle = True)

# Instancio, entreno y testeo modelo No-Optimizado

In [ ]:
#Creo un clasificador
xgb_model = xgb.XGBClassifier(random_state=0, n_estimators=100)
xgb_model

xgb_model.fit(x_train, y_train)
xgb_model_prediction = xgb_model.predict(x_test)

#Evaluacion y visualizacion de resultados
print("Matrix de confusion de TFID:")
print(confusion_matrix(y_test, xgb_model_prediction))

print()

print("TFID Metrics:")
print(metrics.classification_report(y_test, xgb_model_prediction))

Matrix de confusion de TFID:
[[6062 1537]
 [1138 6263]]

TFID Metrics:
              precision    recall  f1-score   support

           0       0.84      0.80      0.82      7599
           1       0.80      0.85      0.82      7401

    accuracy                           0.82     15000
   macro avg       0.82      0.82      0.82     15000
weighted avg       0.82      0.82      0.82     15000



# Busqueda de los mejores hiperParametros

## Con random Search

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 4, 5],
    'min_child_weight': [1, 2, 3],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
}

xgb_model_opt = xgb.XGBClassifier(random_state=0, n_estimators=100)
random_search = RandomizedSearchCV(xgb_model_opt, param_distributions=param_grid, n_iter=50, cv=5, n_jobs=-1, scoring='accuracy')

random_search.fit(x_train, y_train)
best_params = random_search.best_params_


# Evaluar el rendimiento accuracy en el conjunto de prueba
accuracy = random_search.score(x_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy}")

best_model = random_search.best_estimator_

Precisión en el conjunto de prueba: 0.8337333333333333


# Evaluacion y prediccion final

In [ ]:
dft = pd.read_csv("test.csv")
ds_test = dft.copy()

ds_test_ID = ds_test.ID
ds_test_for_predict = ds_test.review_es

dataTest = vectorizer.fit_transform(ds_test_for_predict)

test_prediction = best_model.predict(dataTest)

ds_submission = pd.DataFrame({'ID': ds_test['ID'], 'sentimiento': test_prediction})
ds_submission.to_csv('./prediccion_final_boost.csv', index=False)

joblib.dump(best_model, 'xgboost_entrenado.joblib')

['xgboost_entrenado.joblib']

In [ ]:
print(best_params)

{'subsample': 0.8, 'n_estimators': 300, 'min_child_weight': 1, 'max_depth': 4, 'learning_rate': 0.2, 'colsample_bytree': 1.0}
